In [37]:
import numpy as np
import pandas as pd

import matplotlib.pylab as plt


df = pd.read_csv(u'../data-mining/data/iris.txt',sep=' ')

In [38]:
A = np.hstack([
        np.matrix(df.sl).T, 
        np.matrix(df.sw).T, 
        np.matrix(df.pl).T, 
        np.matrix(df.pw).T])
A[:5] # sample view

matrix([[ 5.1,  3.5,  1.4,  0.2],
        [ 4.9,  3. ,  1.4,  0.2],
        [ 4.7,  3.2,  1.3,  0.2],
        [ 4.6,  3.1,  1.5,  0.2],
        [ 5. ,  3.6,  1.4,  0.2]])

In [39]:
c = np.matrix(df.c).T
x_test = A[1,:] + 5*np.random.randn(1,4)
x_test

matrix([[ 7.36416605,  9.59877946, -3.77513111, -0.90300021]])

In [40]:
import math
def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return math.sqrt(distance)


In [41]:
import operator 
def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [42]:
import operator
def getResponse(neighbors):
    classVotes = []
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

In [45]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [ ]:
split = 0.67
#loadDataset('iris.data', split, trainingSet, testSet)
#print 'Train set: ' + repr(len(trainingSet))
#print 'Test set: ' + repr(len(testSet))
#generate predictions
predictions=[]
k = 10
for x in range(len(x_test)):
    neighbors = getNeighbors(A, x_test[x], k)
    result = getResponse(neighbors)
    predictions.append(result)
    print('> predicted=' + repr(result) + ', actual=' + repr(x_test[x][-1]))
accuracy = getAccuracy(x_test, predictions)
print('Accuracy: ' + repr(accuracy) + '%')